In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from selenium.webdriver.common.keys import Keys
import time
import re
from tqdm.notebook import tqdm

In [2]:
import pandas as pd
import numpy as np
import random
from datetime import datetime ,timedelta , date

### __크롬 드라이버 연동__

In [3]:
my_path = 'D:/downloads/'
driver = webdriver.Chrome(my_path+'chromedriver.exe')
time.sleep(2)

### __페이지 html 소스를 가져오기 전에 필터링을 적용하기 위한 함수들__

__뉴스의 타입을 설정하는 함수 : article , video , slide 중 article 항목만 클릭__

In [4]:
def select_type(driver):
    driver.find_element_by_xpath('//*[@id="wrapper"]/div/div[1]/div/div[2]/div[2]/button').click()
    driver.find_element_by_xpath('//*[@id="wrapper"]/div/div[1]/div/div[2]/div[2]/ul/li[1]/label').click()
    time.sleep(2)
    return

__검색 기간을 설정하는 함수 : 밑에 메인 코드에서 start_date , end_date 를 설정함__

In [5]:
def set_duration(driver): # 검색 기간 설정
    driver.find_element_by_xpath('//*[@id="wrapper"]/div/div[1]/div/div[2]/div[3]/div[1]/div[1]/button').click()
    driver.find_element_by_xpath(f'//*[@id="{start_date[0]}"]').click()

    driver.find_element_by_xpath('//*[@id="wrapper"]/div/div[1]/div/div[2]/div[3]/div[1]/div[2]/button').click()
    driver.find_element_by_xpath(f'//*[@id="{start_date[1]}"]').click()

    driver.find_element_by_xpath('//*[@id="wrapper"]/div/div[1]/div/div[2]/div[3]/div[1]/div[3]/button').click()
    driver.find_element_by_xpath('//*[@id="2020"]').click()
    time.sleep(2)
    
    driver.find_element_by_xpath('//*[@id="wrapper"]/div/div[1]/div/div[2]/div[3]/div[2]/div[1]/button').click()
    driver.find_element_by_xpath(f'/html/body/div/div/div/div/div[1]/div/div[2]/div[3]/div[2]/div[1]/ul/li[{end_date[0]}]').click()

    driver.find_element_by_xpath('//*[@id="wrapper"]/div/div[1]/div/div[2]/div[3]/div[2]/div[2]/button').click()
    driver.find_element_by_xpath(f'/html/body/div/div/div/div/div[1]/div/div[2]/div[3]/div[2]/div[2]/ul/li[{end_date[1]}]').click()

    driver.find_element_by_xpath('/html/body/div/div/div/div/div[1]/div/div[2]/div[3]/div[2]/div[3]/button').click()
    driver.find_element_by_xpath('/html/body/div/div/div/div/div[1]/div/div[2]/div[3]/div[2]/div[3]/ul/li[1]').click()
    time.sleep(2)
    return

__필터 적용한 검색 버튼 클릭 함수__

In [6]:
def filtered_search(driver):
    driver.find_element_by_xpath('//*[@id="wrapper"]/div/div[1]/div/div[1]/div[2]/div/a').click()
    time.sleep(2)
    return

__Load more 버튼을 끝까지 눌러서 모든 기사를 페이지에 로딩하는 함수__ 

In [7]:
def load_all_news(driver):
    while True:
        try:
            driver.find_element_by_xpath('/html/body/div/div/div/div/div[2]/div/div[3]/div[2]/a').click()
            time.sleep(1.5)
        except Exception as E:
            print(E)
            break
    return

__각 기사 태깅 후에 `12 hours ago` 와 같은 정보로 부터 날짜를 계산하는 함수: 현재 시간에서 빼는 형식으로__

In [8]:
def get_date(tag): # 뉴스의 날짜 post date 계산
    global current_time
    
    posted_time_str = articles[1].select('div.info > header > div > span.time')[0].text
    if 'day' in posted_time_str:
        days = re.search('\d+',posted_time_str).group()
        posted_time = current_time - timedelta(days = int(days))
    else:
        hours = re.search('\d+',posted_time_str).group()
        posted_time = current_time - timedelta(hours = int(hours))
    
    return date.isoformat(posted_time)

### __검색 파라미터 설정__

In [9]:
keywords = ['Trump','Biden','2020%20president']
start_date=['10','25']
end_date = ['10','31']
current_time = datetime.now()
data=pd.DataFrame()

### 메인 코드

In [10]:
for keyword in keywords:
    driver.get(f'https://www.foxnews.com/search-results/search?q={keyword}')
    time.sleep(2)

    select_type(driver)
    set_duration(driver)
    filtered_search(driver)
    load_all_news(driver)

    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    articles = soup.select('#wrapper > div > div.search-results-content > div > div.collection.collection-search.active > div > article')

    news = [ {'headlines':article.select('h2')[0].text,
             'date':get_date(article),
             'Press':'Fox',
             'Link':article.select('div.info > header > h2 > a')[0].attrs['href'],
             'keyword':keyword} for article in articles]
    news_df = pd.DataFrame(news)
    data = pd.concat([data,news_df])

Message: element not interactable
  (Session info: chrome=86.0.4240.75)

Message: element not interactable
  (Session info: chrome=86.0.4240.75)

Message: element not interactable
  (Session info: chrome=86.0.4240.75)



In [11]:
data

,headlines,date,Press,Link,keyword
0,"Live Updates: 2020 presidential race, Biden vs...",2020-10-29,Fox,https://www.foxnews.com/politics/live-updates-...,Trump
1,Michael Moore doesn't believe the polls showin...,2020-10-29,Fox,https://www.foxnews.com/media/michael-moore-sa...,Trump
2,Rapper Lil Wayne meets with Trump,2020-10-29,Fox,https://www.foxnews.com/entertainment/rapper-l...,Trump
3,Fox News Poll: Biden's lead over Trump narrows...,2020-10-29,Fox,https://www.foxnews.com/politics/poll-biden-tr...,Trump
4,Golf legend Jack Nicklaus endorses Trump,2020-10-29,Fox,https://www.foxnews.com/sports/golf-legend-jac...,Trump
...,...,...,...,...,...
94,Coal workers fear a Biden presidency will deci...,2020-10-30,Fox,https://www.foxnews.com/us/coal-industry-biden...,2020%20president
95,Philadelphia police budget cuts sought by 4 co...,2020-10-30,Fox,https://www.foxnews.com/politics/philadelphia-...,2020%20president
96,Tucker Carlson: Hunter Biden scandal shows med...,2020-10-30,Fox,http://www.foxnews.com/opinion/tucker-carlson-...,2020%20president
97,"Michigan bars, restaurants will require diners...",2020-10-30,Fox,https://www.foxnews.com/politics/michigan-bars...,2020%20president
